# Защита персональных данных клиентов

**Описание проекта**

Нужно защитить данные клиентов страховой компании «Хоть потоп». Разработать такой метод преобразования данных, чтобы по ним было сложно восстановить персональную информацию. Обосновать корректность его работы.

**Описание данных**

Признаки:

* пол,
* возраст,
* зарплата застрахованного,
* количество членов семьи.

Целевой признак:

* количество страховых выплат клиенту за последние 5 лет.

## Загрузка данных

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score

In [2]:
try:  # local import
    df = pd.read_csv('./datasets/insurance.csv')
except:  # from Praktikum server
    df = pd.read_csv('/datasets/insurance.csv')

In [3]:
df = df.astype('int')
display(df.head())
df.info()

,Пол,Возраст,Зарплата,Члены семьи,Страховые выплаты
0,1,41,49600,1,0
1,0,46,38000,1,1
2,0,29,21000,0,0
3,0,21,41700,2,0
4,1,28,26100,0,0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype
---  ------             --------------  -----
 0   Пол                5000 non-null   int64
 1   Возраст            5000 non-null   int64
 2   Зарплата           5000 non-null   int64
 3   Члены семьи        5000 non-null   int64
 4   Страховые выплаты  5000 non-null   int64
dtypes: int64(5)
memory usage: 195.4 KB


Даннные не содержат пропусков, можно переходить на следующий шаг.

## Умножение матриц

Предсказания:

$$
a = Xw
$$

Задача обучения:

$$
w = \arg\min_w MSE(Xw, y)
$$

Формула обучения:

$$ w = (X^T X)^{-1} X^T y $$

$$ \Leftrightarrow w = X^{-1} (X^T)^{-1} X^T y $$

$$ \Leftrightarrow w = X^{-1} y $$

Обозначения:

- $X$ — матрица признаков (нулевой столбец состоит из единиц)

- $y$ — вектор целевого признака

- $P$ — матрица, на которую умножаются признаки

- $w$ — вектор весов линейной регрессии (нулевой элемент равен сдвигу)

**Подготовка:**

Умножим признаки на обратимую матрицу. Проверим, изменится ли качество линейной регресии.

Для раскрытия скобок и преобразований воспользуемся свойствами матриц:

* Транспонированное произведение матриц равно произведению транспонированных матриц, взятых в обратном порядке:  
    $ (AB)^T = B^T A^T $


* Произведение матриц ассоциативно:  
    $ (AB)C = A(BC) $
    
    
* Свойства обратных и единичных матриц:  
    $ AA^{-1} = A^{-1}A = E $  
    $ AE = EA = A $  
    $ (AB)^{-1} = B^{-1}A^{-1} $  

**Обоснование:**

Умножим матрицу признаков $X$ на произвольную, обратимую матрицу $P$, получим: $a' = XPw'$, где $ w' = ((XP)^T\space XP)^{-1}\space(XP)^T y $

упростим: $ w' = ((XP)^T\space XP)^{-1}\space(XP)^T y $

$ \Leftrightarrow w' = (XP)^{-1}\space((XP)^T)^{-1}\space(XP)^T y $

$ \Leftrightarrow w' = (XP)^{-1}\space(P^T X^T)^{-1}\space P^T X^T y $

$ \Leftrightarrow w' = (XP)^{-1} E y $

$ \Leftrightarrow w' = P^{-1} X^{-1} y $

$ \Leftrightarrow w' = P^{-1} w $

Подставим: $𝑤′$ в формулу: $a'=XPw'$, получим:

$ a'= XP\space P^{-1}w$

$ \Leftrightarrow a'= Xw$

$ \Leftrightarrow a'= a$

**Ответ:** Качество линейной регрессии не изменится, так как $a=a'$ и $w=w'$.

## Алгоритм преобразования

**Алгоритм**

1. Возьмем случайно сгенерированную матрицу $P$ размера $n*n$, определим $n$ по числу признаков в $X-1$;
2. Проверим существование обратной матрицы $P$;
3. Если обратная матрица существует, то умножим исходную матрицу $X$ на матрицу $P$.
4. Для проверки определим метрику R2, и сравним результат на оригинальном и преобразованном датасетах.

**Обоснование**

Согласно доказательству выше, при умножении матрицы признаков $X$ на произвольную, обратимую матрицу $P$, качество линейной регрессии останется неизменным.

In [4]:
def obfuscation(dataset, target, seed=42):
    
    np.random.seed(seed)
    n = dataset.shape[1] - 1
    
    try:
        P = np.random.normal(size=(n, n))
        np.linalg.inv(P)
    except:
        P = np.random.normal(size=(n, n))
        np.linalg.inv(P)
    
    select = dataset.drop(target, axis=1).columns
    
    dataset[select] = dataset[select] @ P
    
    return dataset

## Проверка алгоритма

In [5]:
def ml_run(dataset, target, secret=False):
    
    if secret:
        X = obfuscation(dataset, target).drop(target, axis=1)
    else:
        X = dataset.drop(target, axis=1)
        
    y = dataset[target]
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)
 
    model = LinearRegression()
    model.fit(X_train, y_train)
    prediction = model.predict(X_test)
    
    return r2_score(y_test, prediction)

In [6]:
target = 'Страховые выплаты'

print(f'R2 score on the original dataset: {ml_run(df, target, secret=False):.8f}')
print(f'R2 score on the obfuscated dataset: {ml_run(df, target, secret=True):.8f}')

R2 score on the original dataset: 0.42547785
R2 score on the obfuscated dataset: 0.42547785


**Вывод**

Оценка R2 совпадает до восьмого знака после запятой для моделей линейной регрессии, обученных на оригинальном и обфусцированном датасетах. Таким образом мы экспериментально подтвердили математическое обоснование.